In [1]:
from postgisUtil import postgisUtil
from matplotlib import pyplot as plt
import pickle
import os

In [ ]:
def db_init():
    directory_name = "ASO_Lidar/"
    basin_names  = ["Merced", "Tuolumne"]
    feature_names = ["DEM", "SLP", "ASP", "NOR", "CHM", "VEG"]
    kNN_db = postgisUtil("knn_project", "zeshi")
    kNN_db.create_schema("topo")
    for basin in basin_names:
        print basin
        for i, feature in enumerate(feature_names):
            print feature
            file_name = directory_name + basin + "_500m_" + feature + ".tif"
            if i == 0:
                kNN_db.load_map_to_db(file_name, feature, "topo", basin.lower(), table_exist=False)
            else:
                kNN_db.load_map_to_db(file_name, feature, "topo", basin.lower())
    kNN_db.close()

In [ ]:
db_init()

In [ ]:
def db_geom_init(site_name, schema_exist=True):
    if site_name == "merced":
        site_name_abbr = ""
    if site_name == "tuolumne":
        site_name_abbr = "tb"
    knn_db = postgisUtil("knn_project", "zeshi")
    if not schema_exist:
        knn_db.create_schema('sensors')
    knn_db.create_table(site_name, 'sensors', ['site_id'], ['SERIAL'])
    knn_db.add_geometry_column('sensors', site_name, 'site_coords', 32611, 'POINT', 2, use_typemod='false')
    location = pickle.load(open("kNN_training_testing/" + site_name_abbr + "_sensor_idx.p"))
    test_coords = knn_db.convert_idx_to_coords('DEM', location[1], location[0], 'topo', site_name)
    print test_coords
    knn_db.add_geoms_to_table(test_coords[0], test_coords[1], 32611, 'sensors', site_name, 'site_coords')
    map_pixel_values = knn_db.geoms_table_to_map_pixel_values(['DEM', 'ASP'], 'sensors', site_name, 'site_coords', 'topo', site_name)
    print map_pixel_values
    knn_db.close()

In [11]:
def db_swe_init(site_name, schema_exist=True):
    directory_name = "ASO_Lidar/"
    knn_db = postgisUtil("knn_project", "zeshi")
    if not schema_exist:
        knn_db.create_schema("swe_lidar")
    lidar_fns = os.listdir("ASO_Lidar/")
    if site_name.lower()=="merced":
        site_name_ab = "MB20"
    elif site_name.lower()=="tuolumne":
        site_name_ab = "TB20"
    elif site_name.lower()=="tuolumne_survey":
        site_name_ab = "TB20"
    elif site_name.lower()=="tuolumne_survey_plus":
        site_name_ab = "MBTB20"
    lidar_site_fns = [temp_str for temp_str in lidar_fns if temp_str.startswith(site_name_ab) and temp_str.endswith('500m.tif')]
    if site_name.lower()=="tuolumne_survey_plus":
        lidar_site_fns = [temp_str for temp_str in lidar_fns if temp_str.startswith("TB2016") or temp_str.startswith("MBTB2014")]
    print lidar_site_fns
    lidar_site_date = [temp_str[2:10] for temp_str in lidar_site_fns]
    if site_name.lower()=="tuolumne_survey_plus":
        lidar_site_date = []
        for fn in lidar_site_fns:
            if fn.startswith("TB"):
                lidar_site_date.append(fn[2:10])
            else:
                lidar_site_date.append(fn[4:12])
    for i, lidar_site_fn in enumerate(lidar_site_fns):
        file_name = directory_name + lidar_site_fns[i]
        print file_name
        if i == 0:
            knn_db.load_map_to_db(file_name, lidar_site_date[i], "swe_lidar", site_name, table_exist=False)
        else:
            knn_db.load_map_to_db(file_name, lidar_site_date[i], "swe_lidar", site_name)
    knn_db.close()

In [3]:
db_swe_init("merced")

['MB20140503_500m.tif', 'MB20140512_500m.tif', 'MB20140429_500m.tif', 'MB20140406_500m.tif', 'MB20140423_500m.tif', 'MB20140414_500m.tif', 'MB20140324_500m.tif']
ASO_Lidar/MB20140503_500m.tif
ASO_Lidar/MB20140512_500m.tif
ASO_Lidar/MB20140429_500m.tif
ASO_Lidar/MB20140406_500m.tif
ASO_Lidar/MB20140423_500m.tif
ASO_Lidar/MB20140414_500m.tif
ASO_Lidar/MB20140324_500m.tif


In [4]:
db_swe_init("tuolumne")

['TB20160407_500m.tif', 'TB20160527_500m.tif', 'TB20140428_500m.tif', 'TB20160401_500m.tif', 'TB20140605_500m.tif', 'TB20140323_500m.tif', 'TB20140511_500m.tif', 'TB20160426_500m.tif', 'TB20140531_500m.tif', 'TB20160620_500m.tif', 'TB20140517_500m.tif', 'TB20140502_500m.tif', 'TB20160416_500m.tif', 'TB20140413_500m.tif', 'TB20140407_500m.tif', 'TB20160607_500m.tif', 'TB20140420_500m.tif', 'TB20160613_500m.tif', 'TB20140527_500m.tif']
ASO_Lidar/TB20160407_500m.tif
ASO_Lidar/TB20160527_500m.tif
ASO_Lidar/TB20140428_500m.tif
ASO_Lidar/TB20160401_500m.tif
ASO_Lidar/TB20140605_500m.tif
ASO_Lidar/TB20140323_500m.tif
ASO_Lidar/TB20140511_500m.tif
ASO_Lidar/TB20160426_500m.tif
ASO_Lidar/TB20140531_500m.tif
ASO_Lidar/TB20160620_500m.tif
ASO_Lidar/TB20140517_500m.tif
ASO_Lidar/TB20140502_500m.tif
ASO_Lidar/TB20160416_500m.tif
ASO_Lidar/TB20140413_500m.tif
ASO_Lidar/TB20140407_500m.tif
ASO_Lidar/TB20160607_500m.tif
ASO_Lidar/TB20140420_500m.tif
ASO_Lidar/TB20160613_500m.tif
ASO_Lidar/TB20140527_5

In [6]:
db_swe_init("tuolumne_survey")

['TB20160407_500m.tif', 'TB20160527_500m.tif', 'TB20140428_500m.tif', 'TB20160401_500m.tif', 'TB20140605_500m.tif', 'TB20140323_500m.tif', 'TB20140511_500m.tif', 'TB20160426_500m.tif', 'TB20140531_500m.tif', 'TB20160620_500m.tif', 'TB20140517_500m.tif', 'TB20140502_500m.tif', 'TB20160416_500m.tif', 'TB20140413_500m.tif', 'TB20140407_500m.tif', 'TB20160607_500m.tif', 'TB20140420_500m.tif', 'TB20160613_500m.tif', 'TB20140527_500m.tif']
ASO_Lidar/TB20160407_500m.tif
ASO_Lidar/TB20160527_500m.tif
ASO_Lidar/TB20140428_500m.tif
ASO_Lidar/TB20160401_500m.tif
ASO_Lidar/TB20140605_500m.tif
ASO_Lidar/TB20140323_500m.tif
ASO_Lidar/TB20140511_500m.tif
ASO_Lidar/TB20160426_500m.tif
ASO_Lidar/TB20140531_500m.tif
ASO_Lidar/TB20160620_500m.tif
ASO_Lidar/TB20140517_500m.tif
ASO_Lidar/TB20140502_500m.tif
ASO_Lidar/TB20160416_500m.tif
ASO_Lidar/TB20140413_500m.tif
ASO_Lidar/TB20140407_500m.tif
ASO_Lidar/TB20160607_500m.tif
ASO_Lidar/TB20140420_500m.tif
ASO_Lidar/TB20160613_500m.tif
ASO_Lidar/TB20140527_5

In [12]:
db_swe_init("tuolumne_survey_plus")

['MBTB20140512_500m.tif', 'TB20160407_500m.tif', 'MBTB20140429_500m.tif', 'TB20160527_500m.tif', 'MBTB20140414_500m.tif', 'TB20160401_500m.tif', 'MBTB20140324_500m.tif', 'TB20160426_500m.tif', 'MBTB20140406_500m.tif', 'TB20160620_500m.tif', 'MBTB20140503_500m.tif', 'TB20160416_500m.tif', 'TB20160607_500m.tif', 'TB20160613_500m.tif']
ASO_Lidar/MBTB20140512_500m.tif
ASO_Lidar/TB20160407_500m.tif
ASO_Lidar/MBTB20140429_500m.tif
ASO_Lidar/TB20160527_500m.tif
ASO_Lidar/MBTB20140414_500m.tif
ASO_Lidar/TB20160401_500m.tif
ASO_Lidar/MBTB20140324_500m.tif
ASO_Lidar/TB20160426_500m.tif
ASO_Lidar/MBTB20140406_500m.tif
ASO_Lidar/TB20160620_500m.tif
ASO_Lidar/MBTB20140503_500m.tif
ASO_Lidar/TB20160416_500m.tif
ASO_Lidar/TB20160607_500m.tif
ASO_Lidar/TB20160613_500m.tif


In [ ]:
snow_sensors = [(38.092, -119.43), 
                (38.047, -119.67), 
                (37.873, -119.35), 
                (37.897, -119.257), 
                (37.911, -119.257)]

In [ ]:
snow_surveys = [(38.092, -119.43), 
                (38.047, -119.67), 
                (37.873, -119.35), 
                (37.897, -119.257), 
                (37.911, -119.257),
                (38.083, -119.633),
                (38.15, -119.617),
                (37.837, -119.325)]

In [ ]:
db = postgisUtil("knn_project", "zeshi")
snow_surveys_lon = [s[1] for s in snow_surveys]
snow_surveys_lat = [s[0] for s in snow_surveys]
snow_survey_x_utm11, snow_survey_y_utm11 = db.convert_spatial_ref_sys(snow_surveys_lon, snow_surveys_lat, 4326, 32611)
print snow_survey_x_utm11, snow_survey_y_utm11
db.close()

In [ ]:
db = postgisUtil("knn_project", "zeshi")
for feature in ['DEM', 'SLP', 'ASP', 'NOR', 'VEG', 'CHM']:
    fn = 'ASO_Lidar_500m/MBTB_500m_' + feature + '.tif'
    if feature == 'DEM':
        db.load_map_to_db(fn, feature, 'topo', 'MBTB', table_exist=False)
    else:
        db.load_map_to_db(fn, feature, 'topo', 'MBTB', table_exist=True)
db.close()

In [ ]:
tioga_sensors_feature = np.load('kNN_training_testing/tioga_sensors_feature.npy')
tioga_sensors_idx_y = map(int, tioga_sensors_feature[:, 0])
tioga_sensors_idx_x = map(int, tioga_sensors_feature[:, 1])
print tioga_sensors_idx_y, tioga_sensors_idx_x

In [ ]:
db = postgisUtil("knn_project", "zeshi")
tioga_sensors_coords = db.convert_idx_to_coords('DEM', tioga_sensors_idx_x, tioga_sensors_idx_y, 'topo', 'mbtb')
db.close()
print tioga_sensors_coords

In [ ]:
db = postgisUtil("knn_project", "zeshi")
db.create_table("tuolumne_survey", "sensors", ['site_id'], ['SERIAL'])
db.add_geometry_column('sensors', "tuolumne_survey", 'site_coords', 32611, 'POINT', 2, use_typemod='false')
db.add_geoms_to_table(snow_survey_x_utm11, snow_survey_y_utm11, 32611, 'sensors', "tuolumne_survey", 'site_coords')
db.close()

In [ ]:
survey_plus_x = np.append(snow_survey_x_utm11, tioga_sensors_coords[0])
survey_plus_y = np.append(snow_survey_y_utm11, tioga_sensors_coords[1])
print survey_plus_x, survey_plus_y

In [ ]:
db = postgisUtil("knn_project", "zeshi")
db.create_table("tuolumne_survey_plus", "sensors", ['site_id'], ['SERIAL'])
db.add_geometry_column('sensors', "tuolumne_survey_plus", 'site_coords', 32611, 'POINT', 2, use_typemod='false')
db.add_geoms_to_table(snow_survey_x_utm11, snow_survey_y_utm11, 32611, 'sensors', "tuolumne_survey_plus", 'site_coords')
db.close()